In [1]:
%run 5_model-modules.ipynb
%run 6_output-heads.ipynb

torch.Size([8, 50])
torch.Size([8, 50, 512])
torch.Size([8, 50, 512])
torch.Size([8, 25])
####################################################
torch.Size([8, 50, 512])
torch.Size([8, 50])
torch.Size([8, 25, 512])
torch.Size([8, 25])
torch.Size([8, 25, 512])


C:\Users\princepsnoctis\Desktop\Paragon Roadmap\ParagonNLP2\.venv\Lib\site-packages\torch\nn\functional.py:6044: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [2]:
import torch

In [ ]:
class Model(torch.nn.Module):
    def __init__(
        self,
        sym_len,     # vocabulary size
        max_sam_len, # max size of sample (input)
        max_nam_len, # max size of name (output)
        unit_cat_len,
        tax_cat_len,
        emb_dim=512,
        num_heads=8,
        ff_dim=2048,
        dropout=0.1,
        num_layers=4
    ):
        super().__init__()

        self.encoder_embedding = Embedding(
            sym_len=sym_len,
            max_seq_len=max_sam_len,
            emb_dim=emb_dim
        )

        self.decoder_embedding = Embedding(
            sym_len=sym_len,
            max_seq_len=max_nam_len,
            emb_dim=emb_dim
        )

        self.encoder = Encoder(
            emb_dim=emb_dim,
            num_heads=num_heads,
            ff_dim=ff_dim,
            dropout=dropout,
            num_layers=num_layers
        )

        self.decoder = Decoder(
            emb_dim=emb_dim,
            num_heads=num_heads,
            ff_dim=ff_dim,
            dropout=dropout,
            num_layers=num_layers
        )

        self.multihead = MultiHead(
            sym_len=sym_len,
            unit_cat_len=unit_cat_len,
            tax_cat_len=tax_cat_len,
            emb_dim=emb_dim
        )

    def forward(
            self,
            encoder_tokens,
            decoder_tokens,
            encoder_mask,
            decoder_mask
    ):
        encoder_embedding = self.encoder_embedding(encoder_tokens)
        decoder_embedding = self.decoder_embedding(decoder_tokens)

        encoder_output = self.encoder(encoder_embedding, encoder_mask)

        decoder_output = self.decoder(
            decoder_input=decoder_embedding,
            encoder_output=encoder_output,
            name_mask=decoder_mask,
            sample_mask=encoder_mask
        )

        outputs = self.multihead(
            encoder_output=encoder_output,
            decoder_output=decoder_output,
        )

        return outputs

    def generate_name